# Cropping synthetic data

- What pictures are always misclassified by every model were targeted in this notebook => https://github.com/albertovpd/viu_tfm-deep_vision_classification/blob/synthetic_data_study/src/synthetic_data_pre_study.ipynb
- For that pics, synthetic data were created to increase the volume of them, expecting to have a better performance => https://github.com/albertovpd/viu_tfm-deep_vision_classification/blob/synthetic_data_study/src/gans_diffaug_pytorch_my_dataset.ipynb

Now the output of that is a 7680x4096 mosaic with 30*16 pics, making a total of 480 fake pictures for every class.

That pics have to be cropped and inserted in their corresponding class folder, to train again the models with that and check if there's any improvement.

Bathroom synthetic ouptut:

![alt text](../output/livingroom_fakes000160.png "pic")

Zooming a sample:

![alt](../output/livingroom_fakes000160_1280_5120.png)

In [7]:
import os
from PIL import Image
from itertools import product

- functions

In [8]:
def tile(filename, dir_in, dir_out, d):
    '''
    https://stackoverflow.com/questions/5953373/how-to-split-image-into-multiple-pieces-in-python
    filename: pic
    dir_in: where the pic is located
    dir_out : where cropped pics will be stored
    d : resolution (in my case 264x264)
    '''
    
    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    w, h = img.size
    
    grid = product(range(0, h-h%d, d), range(0, w-w%d, d))
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out = os.path.join(dir_out, f'{name}_{i}_{j}{ext}')
        img.crop(box).save(out)

- start

In [15]:
base_output = "../input/common_misclassifications/mosaics/"
mosaics = sorted(os.listdir(base_output))
folders = ["bathroom_fakes/", "bedroom_fakes/", "dinning_fakes/", "kitchen_fakes/", "livingroom_fakes/"]
paths = dict(zip(folders, mosaics))
paths

{'bathroom_fakes/': 'bathroom_fakes000160.png',
 'bedroom_fakes/': 'bedroom_fakes000200.png',
 'dinning_fakes/': 'dinning_fakes000200.png',
 'kitchen_fakes/': 'kitchen_fakes000160.png',
 'livingroom_fakes/': 'livingroom_fakes000160.png'}

In [17]:
for p in paths:
    
    # create the folder if doesn't exist
    folder = base_output+p
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # split there the corresponding mosaic
    tile(paths[p], base_output, folder, 256)

- Now they have to be merged with the real pics. For each class different folders will be created, in which fake pics will be shuffled before taking 50, 250 and 480 pics. Whit that folders we'll be able to study models performance vs increase of fake pics